In [ ]:
include("../load.jl")

In [ ]:
using CSV, DataFrames, Statistics, Random

In [ ]:
using Distributed
addprocs(5)

In [ ]:
columns = ["Re", "thick", "M", "C_L"];
X = CSV.read(OCT.DATA_DIR * "airfoil/airfoil_X.csv", DataFrame, copycols=true, header=columns, delim=",");
Y = CSV.read(OCT.DATA_DIR * "airfoil/airfoil_Y.csv", DataFrame, copycols=true, header=["C_D"], delim=",");
# Re = Array(range(10000, stop=35000, step=5000));
# thick = [100,110,120,130,140,145];
# M = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9];
# cl = Array(range(0.35, stop=0.70, step=0.05));

In [ ]:
# Plotting some perspectives on the data
using Plots
plt = Plots.plot(X[:,3], X[:,4], Y[:,1], seriestype=:scatter, markersize = 2)
display(plt)

In [ ]:
m = JuMP.Model(with_optimizer(CPLEX_SILENT))
gm = GlobalModel(model = m)
add_variables_from_data!(gm, X) # Adding variables for each data column.
add_variables_from_data!(gm, Y)
bound_to_data!(gm, X)           # Making sure we bound both our free variables and dependent variables 
bound_to_data!(gm, Y)           # test points and observations. 
add_datadriven_constraint(gm, X, Y.C_D, name = "drag polar", dependent_var = gm.model[:C_D])
gm("drag polar")

In [ ]:
learn_constraint!(gm, max_depth = 5, hyperplane_config = (sparsity=:all,))
gm("drag polar").learners[end]

In [ ]:
clear_tree_constraints!(gm)
add_tree_constraints!(gm)
gm.bbls[1].mi_constraints

In [ ]:
# MSE errors 
bbr = gm.bbls[1]
lnr = bbr.learners[end]
println("Log MSE of OCT:", 1- IAI.score(lnr, bbr.X, bbr.Y, criterion=:mse))
# MSE error of global posynomial
Re = exp.(bbr.X[:,"Re"]); thickness = exp.(bbr.X[:,"thick"]); M = exp.(bbr.X[:,"M"]); 
C_L = exp.(bbr.X[:,"C_L"]); C_D = exp.(bbr.Y);
CDp = 0.0470226 .* (Re).^-0.388166 .* thickness.^0.782129 .* (M).^-0.339824 .* (C_L).^0.94829 +
    190.63 .* (Re).^-0.218175 .* thickness.^3.94137 .* (M).^19.2346 .* (C_L).^1.14997 +
    1.62158 .* (Re).^-0.549562 .* thickness.^1.2895 .* (M).^3.03057 .* (C_L).^1.77464 +
    2.91642e-12 .* (Re).^1.18062 .* thickness.^-1.75547 .* (M).^0.105431 .*(C_L).^-1.4407;
CDp = CDp.^(1/1.64722);
MSEposy = sum((log.(C_D)-log.(CDp)).^2)/size(C_D,1)
println("Log MSE of global posynomial: ", MSEposy)